# 훈련 데이터와 테스트 데이터로 분리하기

In [2]:
import urllib.request
import pandas as pd

In [4]:
#IMDB 리뷰 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x123373050>)

In [5]:
df = pd.read_csv("IMDb_Reviews.csv", encoding = "latin1")
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [6]:
print("전체 셈플의 수 : ", len(df))

전체 셈플의 수 :  50000


In [9]:
train_df = df[:25000]
test_df = df[25000:]

In [19]:
train_df.to_csv("train_data.csv", index = False)
test_df.to_csv("test_data.csv", index = False)

# 필드 정의하기

In [22]:
from torchtext import data

In [26]:
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize = str.split,
                  lower = True,
                  batch_first = True,
                  fix_length = 20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)


#sequential : 시퀀스 데이터 여부. (True가 기본값)
#use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
#tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
#lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
#batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
#is_target : 레이블 데이터 여부. (False가 기본값)
#fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.


# 데이터셋 만들기

In [27]:
from torchtext.data import TabularDataset

In [32]:
train_data, test_data = TabularDataset.splits(
    path = ".", train = "train_data.csv", test = "test_data.csv", format = "csv",
    fields = [("text", TEXT), ("label", LABEL)], skip_header = True)

#path : 파일이 위치한 경로.
#format : 데이터의 포맷.
#fields : 위에서 정의한 필드를 지정. 첫번째 원소는 데이터 셋 내에서 해당 필드를 호칭할 이름, 두번째 원소는 지정할 필드.
#skip_header : 데이터의 첫번째 줄은 무시.

In [34]:
print("훈련 샘플의 개수 :", len(train_data))
print("테스트 샘플의 개수 :", len(test_data))

훈련 샘플의 개수 : 25000
테스트 샘플의 개수 : 25000


In [36]:
print(vars(train_data[0]))

{'text': ['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagine', 'a', 'cynic', 'like', 'me

In [37]:
print(train_data.fields.items())

dict_items([('text', <torchtext.data.field.Field object at 0x153f2f110>), ('label', <torchtext.data.field.Field object at 0x153f1a110>)])


# 단어 집합(Vocabulary) 만들기

In [38]:
TEXT.build_vocab(train_data, min_freq = 10, max_size = 10000)

In [40]:
print("단어 집합의 크기 : ",len(TEXT.vocab))

#단어 갯수(10,000) + <unk>(단어 집합에 없는 단어) + <pad>(문장 길이를 동일하게 하기 위한 padding)

단어 집합의 크기 :  10002


# 토치텍스트의 데이터로더 만들기

In [41]:
from torchtext.data import Iterator

In [42]:
batch_size = 5

In [44]:
train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

In [46]:
print("훈련 데이터의 미니 배치 수 : ", len(train_loader))
print("테스트 데이터의 미니 배치 수 : ", len(test_loader))

#25,000(sample 수) / 5(배치 사이트) = 5000

훈련 데이터의 미니 배치 수 :  5000
테스트 데이터의 미니 배치 수 :  5000


In [49]:
batch = next(iter(train_loader))

#첫번째 미니배치

In [50]:
print(type(batch))

#batch(첫번째 미니배치)의 자료형

<class 'torchtext.data.batch.Batch'>


In [52]:
print(batch.text)

#batch의 text필드 출력

tensor([[  37,  100,    0,    9,   14,   82, 1746,    6,    2,  519,    5,    0,
         6760,   50,    9,  202,    0,    0,    9, 1374],
        [ 168,    5,  100,  715, 1874,  125,    0,   47,   31, 5265, 4856,    8,
            2,    0,    0,   31,    2, 1611,    5, 9508],
        [   0,   25,  244,  333,   12,  917,    6,   28,  357,  490,   34,    2,
          558, 4199,   19,  143,    6,    2, 1426,    0],
        [  10,    7,   30,    5,   55,   36,   84,    0,   36,    2,  181,   86,
            3,   87, 1679, 4213,   10,   20,    6,    0],
        [   9,   26,  260,  171,    3,  156, 1386,  493,  860,  128,    2, 1356,
          497,   17, 1173,  847,  860,  154,    8,    0]])


In [53]:
print(batch.text[0])

#batch의 0번째 sample의 text필드 출력

tensor([  37,  100,    0,    9,   14,   82, 1746,    6,    2,  519,    5,    0,
        6760,   50,    9,  202,    0,    0,    9, 1374])
